In [8]:
.include header

In [ ]:
//%%file header/NURBS.h
#pragma once

#include <vector>
#include <iostream>
#include <stdexcept>
#include <memory>
#include <string>
#include <iterator>

#include "eliastocco_namespace_Bspline.h"
#include "knot_vector.h"
#include "dynamic_vector.h"
#include "Bspline_operator.h"


namespace eliastocco {

    namespace polynomial {

        namespace Bspline {        
 
            template < class type , t_dim dimension = 1 , t_dim codimension = 1 >
                class NURBS {

                    #define thisclass NURBS<type,dimension,codimension>    

                    public :
                        typedef std :: array<type,codimension> Type_out;
                        typedef std :: array<type,codimension+1> Type_out_deBoor;
                        typedef std :: array<type,dimension>   Type_in;

                        typedef std :: array<t_dim,dimension>  Index_t;

                        template < t_dim Dim >
                            using template_knots_t = std :: array < knot_vector<type> , Dim>;
                        typedef template_knots_t<dimension> knots_t;
                        
                        typedef std :: array<type,codimension> single_cp_t;
                        template < t_dim Dim >
                            using template_cp_t = container::dynamic::dynamic_vector < Dim , single_cp_t >;


                        typedef template_cp_t<dimension> cp_t;
                    
                        typedef template_cp_t<dimension+1> cp_t_NURBS;
                    
                    
                    
                        typedef container::dynamic::dynamic_vector < dimension , type > weight_t;

//********************************************************************************************************************
                    
                    private :
                        thisclass(){};
                    
                   /* public :
                        thisclass ( const knots_t knots , const cp_t cp , const weight_t weight )
                            :thisclass(knots){
                                _cp = cp ;
                                _weight = weight;
                            };*/
                        
                    public :
                        thisclass ( const knots_t knots , const cp_t cp )
                            :thisclass(knots){
                                _cp = cp ;
                            };
                        
                    //non sto controllando la correttezza degli indici dei control points
                    //non sto controllande che ogni control point abbia la lunghezza giusta
                    //per comodità ho deciso di optare per dei control points std :: valarray
                    public :
                        thisclass ( const knots_t knots )
                            : _kv(knots),_cp(),_weight(){

                                //std :: cout << "constructor" << std :: endl;
                                Index_t init = Index_t();

                                for(auto i=0; i<dimension; ++i){

                                    auto p = _kv[i].p();
                                    auto n = _kv[i].n();
                                    auto k = _kv[i].knots();
                                    auto v = k.size();

                                    init[i]=n;

                                    assert ( std::is_sorted(k.begin(),k.end()) %% "knot vector not sorted" );
                                    assert ( p > 0 %% "wrong polyniomial degree" );
                                    assert ( n > 1 %% "wrong basis cardinality" );
                                    assert ( p+n+1 == v %% "wrong knot vector size" );

                                }

                                //std :: cout << "_cp" << std :: endl;
                                _cp = cp_t (init);
                                
                                if constexpr ( dimension == 1 ){
                                    _weight.resize(_cp.size());
                                    std::fill(_weight.begin(),_weight.end(),1.0);                                                                                                            
                                } else {
                                    
                                    template_knots_t<dimension-1> curve_knot = {} ;
                                    std :: copy ( std :: next (_kv.begin()) , _kv.end() , curve_knot.begin() );
                                    
                                    NURBS<type,dimension-1,codimension> appo (curve_knot);                                                                        
                                    
                                    _weight.resize(_cp.size());
                                    
                                    for(auto i=0; i<_cp.size();++i){
                                        (this->_weight)[i]=appo._weight;
                                    }                                    
                                }
                            
                            };
                            
//********************************************************************************************************************

                    public :
                        Type_out get ( const Index_t index , const bool check = true ) const { 

                            try { 
                                return _cp[index];
                            } catch ( std::out_of_range e ){
                                if ( check ){
                                    std :: cout << "error : Bspline.get" << std :: endl;
                                    std :: cout << e.what()<< std :: endl;  
                                }
                                return Type_out();
                            } 
                            
                        };                    

//********************************************************************************************************************

                    public : // piccola modifica
                        bool set (const Index_t index, const Type_out value , const bool check = true ) {
                            //std :: cout << "set" << std :: endl;
                            try {
                                _cp[index]=value; 
                            } catch ( std::out_of_range e ){  
                                if ( check ){
                                    std :: cout << "error : Bspline.set" << std :: endl;
                                    std :: cout << e.what()<< std :: endl;  
                                    return false;
                                }
                            } 

                            return true;

                        };
                    
//********************************************************************************************************************
                    
                     public :
                        Type_out get_weight ( const Index_t index , const bool check = true ) const { 

                            try { 
                                return _weight[index];
                            } catch ( std::out_of_range e ){
                                if ( check ){
                                    std :: cout << "error : Bspline.get" << std :: endl;
                                    std :: cout << e.what()<< std :: endl;  
                                }
                                return Type_out();
                            } 
                            
                        };   
                    
//********************************************************************************************************************
                    
                     public : // piccola modifica
                        bool set_weight (const Index_t index, const type value , const bool check = true ) {
                            //std :: cout << "set" << std :: endl;
                            try {
                                _weight[index]=value; 
                            } catch ( std::out_of_range e ){  
                                if ( check ){
                                    std :: cout << "error : Bspline.set" << std :: endl;
                                    std :: cout << e.what()<< std :: endl;  
                                    return false;
                                }
                            } 

                            return true;

                        };

//********************************************************************************************************************

                    protected :
                        static int find_k( const type x , const knot_vector<type> t ) {
                            
                            //std :: cout << "indide find_k" << std :: endl;
                            //std :: cout << "size =" << t.size() << std :: endl;
                            int output = -1;
                            for(auto i = 0 ; i < t.size()-1; ++i){
                                //std :: cout << "i: =" << i << std :: endl;
                                if ( t[i] <= x and x < t[i+1] ){ output = i ;}   
                            }

                            return output;

                        };


                    public :
                        Type_out_deBoor deBoor ( const type x ) const {
                            
                            // https://en.wikipedia.org/wiki/De_Boor%27s_algorithm

                            static_assert ( dimension == 1 , "dimension for deBoor algorithm is greater than 1");

                            //
                            auto v = knots();
                            if ( v.size() > 1 ){
                                std :: cout << "deBoor error : v.size()>1" << std :: endl ;
                                throw std::exception ();
                            }  
                            
                            //
                            knot_vector<type> t = v[0];
                            int p = t.p();
                            template_cp_t<1> c = control_points();                                                     
                            
                            //
                            int k = find_k(x,t); 
                            if ( k < 0 ){        
                                std :: cout << "deBoor error : k<0" << std :: endl ;
                                return Type_out();
                            }

                            //
                            std :: array<t_dim,1> init ;
                            init[0]=p+1;
                            template_cp_t<1> d (init);
                            
                            //
                            auto jj=0;
                            for(auto j=0; j!= p+1 ; ++j){
                                if(j+k-p>=0 and j+k-p < c.size() ){
                                    d[j]=c[j+k-p];
                                }else {
                                    d[j] = Type_out();
                                }
                            }

                            //
                            type alpha=type();
                            for(auto r = 1 ; r !=p+1; ++r){
                                for(auto j = p; j!=r-1; --j){
                                    alpha = (x - t[j+k-p]) / (t[j+1+k-r] - t[j+k-p]);
                                    d[j] = (1.0 - alpha) * d[j-1] + alpha * d[j];

                                }
                            }

                            //
                            Type_out_deBoor output;
                            auto out = d[p];                            
                            std::copy(std::begin(out),std::end(out),std::begin(output));
                            return output;


                        };

                    //valutazione del polinomio in x
                    public :
                        Type_out operator () ( const Type_in x ) const {


                                if constexpr ( dimension == 1 ){
                                    
                                    auto a = deBoor(x[0]);                                    
                                    Type_out out;
                                    auto last = *(a.end());
                                    std :: trasform ( std :: next (a.begin()) , a.end() , out.begin() ,
                                                     [&](const auto c ){ return c/last;} );                                    
                                    
                                    return out;
                                    
                                } else {                                   

                                    // https://pages.mtu.edu/~shene/COURSES/cs3621/NOTES/surface/bspline-de-boor.html

                                    //
                                    std :: array<type,dimension-1> x_surf;
                                    std :: copy(std::next(x.begin()),x.end(),x_surf.begin());

                                    //
                                    template_knots_t<dimension-1> curve_knot = {} ;
                                    std :: copy ( std :: next (_kv.begin()) , _kv.end() , curve_knot.begin() );

                                    //
                                    auto m  = _kv[0].n();
                                    
                                    //
                                    template_cp_t<1> surface_cp = {{m}};                                    
                                    Bspline<type,dimension-1,codimension> curve(curve_knot);

                                    //                                    
                                    for(auto i=0; i<m ; ++i){                                       
                                        curve.set_control_points(_cp[i]);                                        
                                        surface_cp[i]=curve(x_surf);
                                    }
                                    
                                    //
                                    template_knots_t<1> sub_kv = {_kv[0]};

                                    //
                                    Bspline<type,1,codimension> curve_final(sub_kv,surface_cp);

                                    //
                                    return curve_final.deBoor(x[0]);

                                }

                        };

                    //vectorized polinomial evaluation
                    public :
                    template < template < class ... > class vect = std :: vector >
                        vect <Type_out> operator () ( const vect < Type_in > x ) const { 
                            vect <Type_out> y (x.size());
                            std :: transform( std :: begin(x), std :: end(x), std :: begin(y),
                                [&]( const auto a ) { return (*this)(a);});   
                            return y;

                        };

//********************************************************************************************************************

                    public :
                        inline auto knots() const { return _kv; }

                    public :
                        inline auto control_points() const { return _cp; }
                    
                    public :
                        auto clear (){                            
                                Index_t init = Index_t();
                                for(auto i=0; i<dimension; ++i){                               
                                    init[i]=_kv[i].n();      
                                }
                                _cp = cp_t (init);
                            
                            return init;
                            
                        };
                        
//********************************************************************************************************************

                    protected :
                        inline void set_knots(const knots_t user_kv ) { _kv = user_kv ; }
                        
                    protected :
                        inline void set_control_points(const cp_t user_cp ) { _cp = user_cp ; }


//********************************************************************************************************************

                    public :
                        cp_t _cp; //control points
                    
                    public :
                        cp_t_NURBS _cp_NURBS; //control points

                    protected :
                        knots_t _kv; //knot vectors
                    
                    protected :
                        weight_t _weight;
                        

                    #undef thisclass
                };

            }

        }

}

In [2]:
%%file header/NURBS.h
#pragma once

#include "Bspline.h"

namespace eliastocco {

    namespace polynomial {
        
        namespace NURBS {
            
            template < class type , t_dim dimension = 1 , t_dim codimension = 1 >
                class NURBS 
                        : public eliastocco :: polynomial :: Bspline :: Bspline<type,dimension,codimension+1> {
                    
                        typedef container::dynamic::dynamic_vector < dimension , type > weight_t;
                        
                        #define motherclass eliastocco::polynomial::Bspline::Bspline<type,dimension,codimension+1>
                        
                        //using motherclass :: Bspline;
                        
                        public : 
                            
                            
                        private :
                            weight_t weight;
                            
                        #undef motherclass
                    
                };
            
        }
        
    }

}

Overwriting header/NURBS.h


In [9]:
#include "NURBS.h"

In [4]:
//using namespace eliastocco::polynomial::NURBS;
using namespace eliastocco :: polynomial :: Bspline;

In [5]:
knot_vector<double> kv = {0,5,{0,1,2,3,4,5}};
std :: array<knot_vector<double>,1> kv_v = {kv};

In [6]:
eliastocco::polynomial::NURBS::NURBS<double> a (kv_v)